# GENERATIVE ADVERSARIAL NEURAL NETWORK

#### General idea

Noise z --> Generator

Generator --> Discriminator --> 0  
Real data --> Discriminator --> 1
  

#### Objective:
Generator - always output 0.5 regardless of the data source (Generator or Real data)

Discriminator - Adjust noise so that generated data resembles real data

#### Architecture

- Generator: 
z --> Leaky ReLU --> tanh

- Discriminator: 
MNIST --> Leaky ReLU --> sigmoid


In [13]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Load the data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Create space for input data


Placeholders for the input data:

- to the **Discriminator**: real_dim

- to the **Generator**: z_dim

In [15]:
def model_inputs(real_dim, z_dim):
    inputs_real = tf.placeholder(tf.float32, (None, real_dim), name='input_real')
    inputs_z = tf.placeholder(tf.float32, (None, z_dim), name='input_z')
    
    return inputs_real, inputs_z

## Build the Generator

A fully connected hidden layer, followed by leaky ReLU and output layer followed by tanh activation function. 

In [23]:
def generator(z, out_dim, n_units=128, reuse=False, alpha=0.01):
    
    with tf.variable_scope('generator', reuse=reuse):
        
        #Hidden layer
        h1 = tf.layers.dense(z, n_units, activation=None)
        h1 = tf.maximum(alpha*h1, h1)
        
        #Output layer
        logits = tf.layers.dense(h1, out_dim, activation=None)
        out = tf.tanh(logits)
        return out

## Build the Discriminator

A fully connected hidden layer, followed by leaky ReLU and output layer followed by tanh activation function. 

In [24]:
def discriminator(x, n_units=128, reuse=False, alpha=0.01):
    
    with tf.variable_scope('discriminator', reuse=reuse):
        
        #Hidden layer
        h1 = tf.layers.dense(x, n_units, activation=None)
        h1 = tf.maximum(alpha*h1, h1)
        
        #Output layer
        logits = tf.layers.dense(h1, 1, activation=None)
        
        out = tf.sigmoid(logits)
        return out, logits

## Set the hyperparameters

In [36]:
# Input_size: flattened image vector. MNIST consists of images 28x28 pxl 
input_size = 784

# Configurable noise vector
z_size = 100

# Hidden layers' size  
d_hidden_size = 128
g_hidden_size = 128

# Leak factor for leaky ReLU
alpha = 0.01

# Real images labels smoothing factor (for better generalization)
smooth = 0.1

# Learning rate
learning_rate = 0.002

## Build the network

In [26]:
tf.reset_default_graph()

# Input to Generator and Discriminator prep
input_real, input_z = model_inputs(input_size, z_size)

# Generator
g_model = generator(input_z, input_size, n_units=g_hidden_size, alpha=alpha)

# Discriminator input from real dataset  
d_model_real, d_logits_real = discriminator(input_real, n_units=d_hidden_size, alpha=alpha)

# Discriminator input from Generator 
d_model_fake, d_logits_fake = discriminator(g_model, reuse=True, n_units=d_hidden_size, alpha=alpha)

## Calculate losses

In [33]:
# Discriminator loss

labels_real_d = tf.ones_like(d_logits_real) * (1 - smooth)

d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                            logits=d_logits_real, labels=labels_real_d))

labels_fake = tf.zeros_like(d_logits_fake)

d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                             logits=d_logits_fake, labels=labels_fake))

d_loss = d_loss_real + d_loss_fake

# Generator loss

labels_g = tf.ones_like(d_logits_fake)

g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
                         logits=d_logits_fake, labels=labels_g))

## Optimizers

In [37]:
t_vars = tf.trainable_variables()
g_vars = [var for var in t_vars if var.name.startswith('generator')]
d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

d_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
g_train_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)

## Training

In [38]:
batch_size = 100
epochs = 100
samples= []
losses = []

saver = tf.train.Saver(var_list=g_vars)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            batch = mnist.train.next_batch(batch_size)
            
            # Get images, reshape and rescale to pass to Discriminator 
            batch_images = batch[0].reshape((batch_size, 784))
            batch_images = batch_images*2-1
            
            # Sample random noise for Generator 
            batch_z = np.random.uniform(-1, 1, size=(batch_size, z_size))
        
        train_loss_d = sess.run(d_loss, {input_z: batch_z, input_real: batch_images})
        train_loss_g = g_loss.eval({input_z: batch_z}) 

        # Print and save losses
        print("Epoch {}/{}...".format(e+1, epochs),
              "Discriminator Loss: {:.4f}...".format(train_loss_d),
              "Generator Loss: {:.4f}".format(train_loss_g))    
        
        losses.append((train_loss_d, train_loss_g))

Epoch 1/100... Discriminator Loss: 1.7805... Generator Loss: 0.6766
Epoch 2/100... Discriminator Loss: 1.7949... Generator Loss: 0.6857
Epoch 3/100... Discriminator Loss: 1.7621... Generator Loss: 0.7003
Epoch 4/100... Discriminator Loss: 1.8060... Generator Loss: 0.7052
Epoch 5/100... Discriminator Loss: 1.8066... Generator Loss: 0.6931
Epoch 6/100... Discriminator Loss: 1.8327... Generator Loss: 0.6938
Epoch 7/100... Discriminator Loss: 1.8272... Generator Loss: 0.6879
Epoch 8/100... Discriminator Loss: 1.7883... Generator Loss: 0.6933
Epoch 9/100... Discriminator Loss: 1.8294... Generator Loss: 0.6825
Epoch 10/100... Discriminator Loss: 1.8930... Generator Loss: 0.7065
Epoch 11/100... Discriminator Loss: 1.8345... Generator Loss: 0.6883
Epoch 12/100... Discriminator Loss: 1.8514... Generator Loss: 0.7030
Epoch 13/100... Discriminator Loss: 1.7937... Generator Loss: 0.7171


KeyboardInterrupt: 